In [1]:
#polygon.io API Key
import config
api_key = config.api_key

In [20]:
#lib
import pandas as pd
import requests as req
import datetime
from pandas_market_calendars import get_calendar

In [ ]:
#ISSUE:
#1 lengh limit - only 2 years data
#2 call limit - only 5 calls / min
#3 query limit - Max 50000 data points
    #50000 minute = 0.9493 years
    #50000 hours = 5.7077 years
    #50000 hours = 136.9863 years
    

#symbol: stock symbol
#date_A: from date
#date_B: to date
#timespan: minute, hour, day, week, month, quarter, year

In [135]:
#test using VOO
day1 = '2010-01-01'
day2 = datetime.date.today().strftime('%Y-%m-%d')
timespan = 'day'
symbol = 'VOO'

In [132]:
def api_call(symbol, date_A, date_B, timespan, api_key):
    multiplier = 1
    adjusted = True     #default
    sort = 'asc'        #ascending order (oldest at top)
    limit = 50000       #default 5000, must be <50000
    #request url
    url = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/{timespan}/{date_A}/{date_B}?adjusted={adjusted}&sort={sort}&limit={limit}&apiKey={api_key}'
    #requesting
    data = req.get(url).json()
    return data

In [133]:
def json_to_df(data):
    results = data['results']
    df = pd.DataFrame(columns=['time', 'open', 'high', 'low', 'close', 'volume'])
    for result in results:
        df = df.append({
            'time': pd.to_datetime(result['t'], unit='ms'),
            'open': result['o'],
            'high': result['h'],
            'low': result['l'],
            'close': result['c'],
            'volume': result['v']
        }, ignore_index=True)
    return df

In [123]:
#limit checker
def overlimit(day_A, day_B, StockExchange, timespan, limit = 50000):
    
    calendar = get_calendar(StockExchange).schedule(day_A, day_B)
    
    #total trading day
    trading_days = len(calendar)
    
    #total trading hours and minutes
        #stock exchanges with lunch break
    if 'break_start' in calendar.columns.values.tolist():
        calendar['hours'] = ((calendar.market_close - calendar.market_open)-(calendar.break_end - calendar.break_start)) / pd.Timedelta(hours=1) 
        calendar['minutes'] = ((calendar.market_close - calendar.market_open)-(calendar.break_end - calendar.break_start)) / pd.Timedelta(minutes=1) 
        #stock exchanges without lunch break
    else:
        calendar['hours'] = (calendar.market_close - calendar.market_open) / pd.Timedelta(hours=1)
        calendar['minutes'] = (calendar.market_close - calendar.market_open) / pd.Timedelta(minutes=1) 
    trading_hours = calendar['hours'].sum()
    trading_minutes = calendar['minutes'].sum()
    
    #counts
    if timespan == 'day':
        counts = trading_days
    if timespan == 'hour':
        counts  = trading_hours
    elif timespan == 'minute':
        counts = trading_minutes
    else:
        counts = 0
    
    #check
    if counts > limit:
        check = True
    else:
        check = False
        
    return check

#return True if over limit

In [ ]:
#task assigner

In [ ]:
#data intergration